# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I (i.e., WeatherPy) to a DataFrame

In [19]:
city_data_df = pd.read_csv("output_data\weather_data.csv")
city_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tuktoyaktuk,69.4541,-133.0374,-28.48,75,17,3.87,CA,1611975600
1,1,Busselton,-33.6500,115.3333,80.56,47,0,6.53,AU,1611975600
2,2,Albany,42.6001,-73.9662,7.84,68,23,9.28,US,1611975600
3,3,Hermanus,-34.4187,19.2345,66.94,80,75,3.29,ZA,1611975600
4,4,Tanout,14.9709,8.8879,60.46,29,3,13.15,NE,1611975600
...,...,...,...,...,...,...,...,...,...,...
517,517,Shingū,33.7333,135.9833,46.54,51,20,10.13,JP,1611975600
518,518,Piteå,65.3172,21.4794,2.14,89,94,11.27,SE,1611975600
519,519,Katangli,51.7102,143.2326,0.63,92,100,15.90,RU,1611975600
520,520,Canutama,-6.5339,-64.3831,74.89,96,100,3.15,BR,1611975600


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = city_data_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
narrowed_city_df = city_data_df.loc[(city_data_df["Wind Speed"] <= 10) & (city_data_df["Cloudiness"] == 0) & \
                                   (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,Bubaque,11.2833,-15.8333,78.31,68,0,8.77,GW,1611975600
85,85,Minbu,20.1833,94.8833,75.60,32,0,3.38,MM,1611975600
164,164,Sauce,-34.6469,-56.0628,71.06,68,0,7.90,UY,1611975600
376,376,Monywa,22.1167,95.1333,73.44,58,0,7.40,MM,1611975600
422,422,Malanville,11.8685,3.3899,71.49,22,0,7.76,BJ,1611975600
484,484,Edd,13.9297,41.6925,74.16,37,0,9.19,ER,1611975600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Bubaque,GW,11.2833,-15.8333,
85,Minbu,MM,20.1833,94.8833,
164,Sauce,UY,-34.6469,-56.0628,
376,Monywa,MM,22.1167,95.1333,
422,Malanville,BJ,11.8685,3.3899,
484,Edd,ER,13.9297,41.6925,


In [23]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Bubaque,GW,11.2833,-15.8333,Le Mango Lodge
85,Minbu,MM,20.1833,94.8833,Man Thi Tar Hotel Minbu
164,Sauce,UY,-34.6469,-56.0628,NaN
376,Monywa,MM,22.1167,95.1333,Win Unity Hotel
422,Malanville,BJ,11.8685,3.3899,Hotel Sota
484,Edd,ER,13.9297,41.6925,NaN


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))